In [381]:
include("/Users/johnbuckner/.julia/dev/UniversalDiffEq.jl/src/UniversalDiffEq.jl")

Main.UniversalDiffEq

In [385]:
using DataFrames
data = DataFrame(time = 1:100, x1 = rand(100))
data.series .= 1 
for i in 2:20
    data_i = DataFrame(time = 1:100, x1 = rand(100))
    data_i.series .= i
    data = vcat(data,data_i)
end
nothing

In [386]:
# time_column_name = "time", series_column_name = "series"
mod = UniversalDiffEq.MultiCustomDifference(data, (x,i,p,t) -> [0.5], (a = 0.0, ), l=1000)
train = mod -> UniversalDiffEq.train!(mod, loss_function = "conditional likelihood", optimizer = "BFGS",
                        loss_options = (observation_error  = 0.1, process_error = 1e7,),verbose = false)
k = 5
nothing

In [387]:
UniversalDiffEq.leave_site_out(mod, train)

0.2521703832457592

In [388]:
T = 99
pred = preds[1].x1
sample_ = tst.x1
N = length(preds)
score = 0
for t in 1:T
    for i in 1:N
        pred = preds[i].x1
        score += 1/N * sum(sqrt((pred[t]- sample_[t])^2))
        for j in 1:N
            pred_j = preds[j].x1
            score += -1/(2*N^2) * sum(sqrt((pred[t] - pred_j[t])^2))
        end
    end
end 

score/T

0.33861744651821757